In [34]:
import pandas as pd 
import numpy as np
import math

raw_design = pd.read_csv("MEMC_100.csv")
feature_map = [("p_i", "Pinternal"), 
               ("p_s","Pswitch"),
               ("p_l", "Pleak"), 
               ("t_min", "Tarrival"), 
               ("t_max", ["Tarrival", "Ttransition"]), 
               ("x_min", "x"), 
               ("x_max", ["x", "w"]), 
               ("y_min", "y"),
               ("y_max", ["y", "h"]), 
               ("r_tog", "TCoutput"), 
               ("ir", "IR-drop")]

data_features = pd.DataFrame()
for feat_name, raw_name in feature_map:
    if ( type(raw_name) == list):  
        data_features[feat_name] = sum([raw_design[raw_subname] for raw_subname in raw_design[raw_name]])
    else: 
        data_features[feat_name] = raw_design[raw_name]

data_features.shape

(223829, 11)

In [35]:
data_features

,p_i,p_s,p_l,t_min,t_max,x_min,x_max,y_min,y_max,r_tog,ir
0,4.571610e-08,0.000000e+00,1.733237e-08,35000.0,35000.0,205.96,206.72,99.82,101.22,0.0,0.711057
1,0.000000e+00,0.000000e+00,1.733237e-08,0.0,0.0,264.48,265.24,150.22,151.62,0.0,0.658630
2,0.000000e+00,0.000000e+00,1.714345e-08,0.0,0.0,222.68,223.44,144.62,146.02,0.0,0.714736
3,2.851789e-08,4.458124e-07,1.733237e-08,35000.0,35000.0,151.24,152.00,158.62,160.02,0.0,0.688439
4,4.566527e-08,0.000000e+00,1.714345e-08,35000.0,35000.0,143.45,144.21,164.22,165.62,0.0,0.675305
...,...,...,...,...,...,...,...,...,...,...,...
223824,1.236674e-07,0.000000e+00,7.684486e-08,35000.0,35000.0,945.82,951.33,1007.02,1008.42,5.0,0.640829
223825,1.124357e-07,0.000000e+00,7.684486e-08,0.0,0.0,989.71,995.22,1007.02,1008.42,2.0,0.603829
223826,8.439611e-08,0.000000e+00,7.684486e-08,35000.0,35000.0,984.77,990.28,1007.02,1008.42,3.0,0.627079
223827,9.127899e-08,2.472818e-08,7.684486e-08,35000.0,35000.0,1007.76,1013.27,1007.02,1008.42,3.0,0.649148


In [36]:
data_features.to_numpy()

array([[4.57160958e-08, 0.00000000e+00, 1.73323702e-08, ...,
        1.01220000e+02, 0.00000000e+00, 7.11056709e-01],
       [0.00000000e+00, 0.00000000e+00, 1.73323702e-08, ...,
        1.51620000e+02, 0.00000000e+00, 6.58630133e-01],
       [0.00000000e+00, 0.00000000e+00, 1.71434529e-08, ...,
        1.46020000e+02, 0.00000000e+00, 7.14736223e-01],
       ...,
       [8.43961061e-08, 0.00000000e+00, 7.68448558e-08, ...,
        1.00842000e+03, 3.00000000e+00, 6.27079010e-01],
       [9.12789915e-08, 2.47281768e-08, 7.68448558e-08, ...,
        1.00842000e+03, 3.00000000e+00, 6.49148285e-01],
       [1.51094980e-07, 0.00000000e+00, 7.68448558e-08, ...,
        1.00842000e+03, 2.00000000e+00, 6.31218314e-01]])

In [37]:
from tqdm.auto import tqdm

def Decomp(features: np.ndarray, W: int, H: int, C: int, T: int, l: int,
            t: int) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    
    w, h, N = math.ceil(W/l), math.ceil(H/l), math.ceil(T/t)
    P_i, P_s, P_sca, P_all = np.zeros((4,w,h))
    P_t = np.zeros((N,w,h))

    for c in tqdm(range(C)):

        c_features = features[c]
        p_sca = (c_features[0] + c_features[1]) * c_features[9] + c_features[2]
        p_all = c_features[0] + c_features[2] + c_features[1]
        x_n, x_x = math.floor(c_features[5]/l), math.ceil(c_features[6]/l)
        y_n, y_x = math.floor(c_features[7]/l), math.ceil(c_features[8]/l)
        s = (x_x - x_n) * (y_x - y_n)

        M = np.zeros((w,h)) 
        M[x_n-1:x_x][y_n-1:y_x] = 1

        P_i += M * c_features[0]/s
        P_s += M * c_features[1]/s
        P_sca += M * p_sca/s
        P_all += M * p_all/s
        for j in range(N):
            if c_features[3] < j * t and c_features[4] > j * t :
                P_t[j] += M * p_sca/s

    return P_t, P_i, P_s, P_sca, P_all

In [80]:
from tqdm.auto import tqdm

def Decomp(features: pd.DataFrame, W: int, H: int, C: int, T: int, l: int,
            t: int) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    
    w, h, N = math.ceil(W/l), math.ceil(H/l), math.ceil(T/t)
    P_i, P_s, P_sca, P_all = np.zeros((4,w,h))
    P_t = np.zeros((N,w,h))

    for c in tqdm(range(C)):

        c_features = features.iloc[c]
        p_sca = (c_features["p_i"] + c_features["p_s"]) * c_features["r_tog"] + c_features["p_l"]
        p_all = c_features["p_i"] + c_features["p_l"] + c_features["p_s"]
        x_n, x_x = math.floor(c_features["x_min"]/l), math.ceil(c_features["x_max"]/l)
        y_n, y_x = math.floor(c_features["y_min"]/l), math.ceil(c_features["y_max"]/l)
        s = (x_x - x_n) * (y_x - y_n)

        M = np.zeros((w,h)) 
        M[x_n-1:x_x, y_n-1:y_x] = 1
        P_i += M * c_features["p_i"]/s
        P_s += M * c_features["p_s"]/s
        P_sca += M * p_sca/s
        P_all += M * p_all/s
        for j in range(N):
            if c_features["t_min"] < j * t and c_features["t_max"] > j * t :
                P_t[j] += M * c_features["p_sca"]/s

    return P_t, P_i, P_s, P_sca, P_all

In [82]:
P_t, P_i, P_s, P_sca, P_all = Decomp(
    data_features, 
    math.ceil(max(data_features["x_max"])), 
    math.ceil(max(data_features["y_max"])), 
    1000,
    # data_features.shape[0],
    35000,
    1,
    700
    )

  0%|          | 0/1000 [00:00<?, ?it/s]

In [60]:
print(P_t.size / np.count_nonzero(P_i))

ZeroDivisionError: division by zero